## convert to be able to consume it with arcGIS JS

- convert pixel back to raster EPSG coordinates -> lat long


In [1]:
import boto3 
boto3.setup_default_session(profile_name='hawaii')
s3_client = boto3.client('s3')
s3_resource = boto3.resource('s3')
bucket_name = 'hawaii-marine-debris'
version='v3'

In [10]:
# automatically reload modules when they have changed
%load_ext autoreload
%autoreload 2
def download_from_s3(s3_key):
    basename = os.path.basename(s3_key)
    local_file_path = local_tmp_folder.format(basename)
    s3_resource.Bucket(bucket_name).download_file(s3_key, local_file_path)
    return local_file_path

def convert_tif_to_jpeg(tiff_image_path):
    src = rasterio.open(tiff_image_path)
    jpg_path = tiff_image_path.replace(".tif", ".jpg")
    data = src.read()
    profile = src.profile
    result = data.astype(rasterio.uint8, casting='unsafe', copy=False)
    profile.update(driver='jpeg')
    with rasterio.open(jpg_path, 'w', **profile) as dst:
        dst.write(result)
    return jpg_path
                
def upload_to_s3(local_file_path, s3_path,ExtraArgs=None):
    s3_client.upload_file(local_file_path, bucket_name, s3_path,ExtraArgs)

def find_min_xy(coordinates):
    x0 = min([i[0] for i in coordinates])
    x1 = max([i[0] for i in coordinates])
    y0 = min([i[1] for i in coordinates])
    y1 = max([i[1] for i in coordinates])
    return x0,y0,x1,y1


def get_pixel_coordinates(image_path, coordinates):
    driver = gdal.GetDriverByName('GTiff')
    dataset = gdal.Open(image_path)
    band = dataset.GetRasterBand(1)

    cols = dataset.RasterXSize
    rows = dataset.RasterYSize

    transform = dataset.GetGeoTransform()

    xOrigin = transform[0]
    yOrigin = transform[3]
    pixelWidth = transform[1]
    pixelHeight = -transform[5]

    data = band.ReadAsArray(0, 0, cols, rows)

    x0, y0, x1, y1 = coordinates
    x0p = int((x0 - xOrigin) / pixelWidth)
    y0p = int((yOrigin - y0 ) / pixelHeight)
    x1p = int((x1 - xOrigin) / pixelWidth)
    y1p = int((yOrigin - y1 ) / pixelHeight)
    return x0p, y0p, x1p, y1p

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
from pyproj import Proj, transform
# 32611
#26965 ?
inProj = Proj(init='epsg:26904')
outProj = Proj(init='epsg:4326')

x1, y1 = 390500.667334294, 2432788.8433446772
#expected to be something like 
long, lat = transform(inProj,outProj,x1,y1)
print(long,lat)

-160.06075380943668 21.996225452843184


In [12]:
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import boto3
import pandas as pd
import os
root_folder = os.getcwd()
root_folder = "/tmp"
print(root_folder)
image_root = '/tmp/data/images/niihau'

/tmp


In [13]:
def get_jpg_key(tiff_s3_path):
    return "niihau/jpg/"+os.path.basename(tiff_s3_path).replace(".tif",".jpg")

In [14]:
import json
geo_json={}

with open("{}/niihau_tileindex.geojson".format(root_folder)) as f:
    geo_json = json.load(f)
len(geo_json['features'])
tiles_features = geo_json['features']
tiles_features[:2]

#simplify to create what we care
# for each jpg -> original x y coordinates
coords_lookup = []
for idx, tile in enumerate(tiles_features):    
    
    key = get_jpg_key(tile['properties']['s3_path'])
    x1,y1,x2,y2 = find_min_xy(tile['geometry']['coordinates'][0]) 
    coords_lookup.append((key,x1,y1,x2,y2))

In [15]:
len(coords_lookup)

116158

In [16]:
# coords_lookup

# import csv

# with open('tile_coordinates.csv','w') as out:
#     csv_out=csv.writer(out)
#     csv_out.writerow(['s3_key','x1', 'y1','x2','y2'])
#     for row in coords_lookup:
#         csv_out.writerow(row)

In [17]:
coords_lookup_df = pd.read_csv('tile_coordinates.csv')
coords_lookup_df['coord_width'] = coords_lookup_df.apply(lambda x: x.x2-x.x1, axis=1)
coords_lookup_df['coord_height'] = coords_lookup_df.apply(lambda x: x.y2-x.y1, axis=1)
coords_lookup_df[:5]

,s3_key,x1,y1,x2,y2,coord_width,coord_height
0,niihau/jpg/1_10_07_01.jpg,375494.86,2409223.12,375514.86,2409243.12,20.0,20.0
1,niihau/jpg/1_10_07_02.jpg,375510.86,2409223.12,375530.86,2409243.12,20.0,20.0
2,niihau/jpg/1_10_08_01.jpg,375494.86,2409207.12,375514.86,2409227.12,20.0,20.0
3,niihau/jpg/1_10_08_02.jpg,375510.86,2409207.12,375530.86,2409227.12,20.0,20.0
4,niihau/jpg/1_10_08_03.jpg,375526.86,2409207.12,375546.86,2409227.12,20.0,20.0


In [22]:
# s3://hawaii-marine-debris/ml/result/v3/all_debris_detected_2.csv

detected_debris_key = "ml/result/{}/all_debris_detected_from_all.csv".format(version)
debris_path = "{}/all_debris_detected_with_all.csv.csv".format(root_folder)
s3_resource.Bucket(bucket_name).download_file(detected_debris_key, debris_path)
all_debris_df = pd.read_csv(debris_path)
all_debris_df[:5]

evaluation_key = "ml/result/{}/evaluation_result.csv".format(version)
evaluation_path = "{}/evaluation_result.csv".format(root_folder)
s3_resource.Bucket(bucket_name).download_file(evaluation_key, evaluation_path)

test_key = "ml/{}/data/keras_annotations_test.csv".format(version)
test_path = "{}/keras_annotations_test.csv".format(root_folder)
s3_resource.Bucket(bucket_name).download_file(test_key, test_path)
# s3://hawaii-marine-debris/niihau/deduped_annotations.csv
#may contain alot of overlap to each other
print(len(all_debris_df[all_debris_df['score']>0.3]))

good_enough_debris_set = all_debris_df[all_debris_df['score']>0.3]

In [24]:
good_enough_debris_set = good_enough_debris_set.drop(['Unnamed: 0'],axis=1)
debris_row = good_enough_debris_set.iloc[0]
image_cords_row = coords_lookup_df[coords_lookup_df['s3_key']==debris_row.image_s3_key].iloc[0]

In [37]:
image_row = image_cords_row

In [85]:
label_path = "/tmp/labels_{}.csv".format(version)
s3_resource.Bucket(bucket_name).download_file('ml/{}/data/labels.csv'.format(version), label_path)
label_df = pd.read_csv(label_path,names=['label','id'])
label_df
label_lookup = label_df.set_index('id').T.to_dict('records')[0]
label_lookup 
good_enough_debris_set['label'] = good_enough_debris_set.apply(lambda row: label_lookup[row.label], axis=1)

In [38]:
(debris_row.y2/1000)*image_row.coord_height

16.78

In [39]:

from pyproj import Proj, transform
from datetime import datetime, date, timedelta

inProj = Proj(init='epsg:26904')
outProj = Proj(init='epsg:4326')

def get_relative_coord(image_row, debris_row, image_size=(1000,1000)):
#     x1 = round(image_row.x1 + ((debris_row.x1/1000)*image_row.coord_width),2)
#     x2 = round(image_row.x1 + ((debris_row.x2/1000)*image_row.coord_width),2)
#     y1 = round(image_row.y1 + ((debris_row.y1/1000)*image_row.coord_height),2)
#     y2 = round(image_row.y1 + ((debris_row.y2/1000)*image_row.coord_height),2)
    x1 =image_row.x1 + ((debris_row.x1/1000)*image_row.coord_width)
    x2 = image_row.x1 + ((debris_row.x2/1000)*image_row.coord_width)
    y1 = image_row.y1 + ((debris_row.y1/1000)*image_row.coord_height)
    y2 = image_row.y1 + ((debris_row.y2/1000)*image_row.coord_height)
    long, lat = transform(inProj,outProj,x1,y1)
#     long, lat = transform(inProj,outProj,((x1+x2)/2), ((y1+y2)/2))
    #size needs better calculations for plot.
    width, height = round((x2-x1),2), round((y2-y1),2)
    return lat, long, width, height

get_relative_coord(image_cords_row,debris_row)

(21.80500991977527, -160.23916115359916, 0.78, 0.8)

In [40]:
good_enough_debris_set[:5]

,image_s3_key,x1,y1,x2,y2,label,score,overlapped_test_index,overlapped_with_training_index
11,niihau/jpg/1_1130_16_26.jpg,305,672,344,711,7,0.523061,644.0,644.0
12,niihau/jpg/1_1130_16_26.jpg,313,678,353,718,7,0.332871,644.0,644.0
21,niihau/jpg/1_1130_16_27.jpg,774,288,814,329,13,0.381062,NaN,NaN
26,niihau/jpg/1_1171_04_25.jpg,265,437,304,476,7,0.439906,NaN,2256.0
27,niihau/jpg/1_1171_04_25.jpg,319,521,360,562,6,0.344256,156.0,156.0


In [41]:
# -160.1793628159, 21.8342278972
max(good_enough_debris_set.index)

25320

In [42]:
debris_lat_long_rows = []
for idx, debris_row in good_enough_debris_set.iterrows():
    image_cords_row = coords_lookup_df[coords_lookup_df['s3_key']==debris_row['image_s3_key']].iloc[0]
    lat, long, width, height = get_relative_coord(image_cords_row,debris_row)
    time = datetime.now().strftime("%Y-%m-%d")
    debris_lat_long_rows.append((time, lat, long, width, height,debris_row.label))
    print(" pct done -- {0:.2f}".format(idx/max(good_enough_debris_set.index)),end='\r')
    
    

In [43]:
with open('debris_lat_long_coordinates.csv','w') as out:
    csv_out=csv.writer(out)
    csv_out.writerow(['time','lat','long','width','height','debris_type'])
    for row in debris_lat_long_rows:
        csv_out.writerow(row)

In [44]:
upload_to_s3('debris_lat_long_coordinates.csv', 'niihau/map/debris_lat_long_coordinates.csv', ExtraArgs={'ACL':'public-read'})

In [50]:

x1 = round(image.x1 + ((debris_row.x1/1000)*image_row.coord_width),10)
x2 = round(image.x1 + ((debris_row.x2/1000)*image_row.coord_width),10)
y1 = round(image.y1 + ((debris_row.y1/1000)*image_row.coord_height),10)
y2 = round(image.y1 + ((debris_row.y2/1000)*image_row.coord_height),10)
# min_lat_long = 
# calculate center_lat_long and size.


# # 32611
# #26965 ?
# x1, y1 = 390500.667334294, 2432788.8433446772
# #expected to be something like 
# long, lat = transform(inProj,outProj,x1,y1)
# print(long,lat)

375500.96

In [21]:
all_key = "niihau/niihau_tileindex.csv"
all_path = "{}/niihau_tileindex.csv".format(root_folder)
s3_resource.Bucket(bucket_name).download_file(all_key, all_path)
all_tiles_df = pd.read_csv(all_path)
all_tiles_df[:5]

,tile_name,directory,s3_path
0,1_10_07_01.tif,1_10,https://s3-us-west-2.amazonaws.com/hawaii-mari...
1,1_10_07_02.tif,1_10,https://s3-us-west-2.amazonaws.com/hawaii-mari...
2,1_10_08_01.tif,1_10,https://s3-us-west-2.amazonaws.com/hawaii-mari...
3,1_10_08_02.tif,1_10,https://s3-us-west-2.amazonaws.com/hawaii-mari...
4,1_10_08_03.tif,1_10,https://s3-us-west-2.amazonaws.com/hawaii-mari...


5657
